In [1]:
import pandas as pd
import numpy as np
import random
import shutil

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sn


import os
import glob
from pathlib import Path

from collections import Counter
from itertools import cycle, islice

from PIL import Image
from sys import argv
from warnings import filterwarnings
filterwarnings("ignore")

C:\Users\MICHEAL\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
pat = "annotated_batches"
#pat = "post_loss_annotated"
csvs = f"annotations_CSV"
xmls = f"annotations_XML"
yolos = f"annotations_YOLO"

if not os.path.isdir(f"{pat}/cleaned_annotations_CSV"):   #create batch folder if it does not exist
        os.makedirs(f"{pat}/cleaned_annotations_CSV")
new_csvs = f"{pat}/cleaned_annotations_CSV"


path_csv = f"{csvs}/*?.csv"

path_xml = f"{xmls}/*?.xml"
path_yolo = f"{yolos}/*?.txt"

In [3]:
def path_to_list(path, file_type):
    """
    function for extracting files name from either of xml or yolo directory
    input
    path: path to xml or yolo directory
    file_type: - specification of either txt or yolo"""
    path_list = []
    for files in glob.glob(path):  #iterate through folder
        f_name = os.path.split(files)[-1]
        f = f_name.split(f".{file_type}")[0]
        path_list.append(files)
        #print(f)
    #print(len(path_list))
    return path_list

In [4]:
i = 0
for files in glob.glob(path_csv): #erate through folder to concatenate csv into dataframes
    #print(files)
    #extract batch names
    #f_name = os.path.split(files)[-1]
    #batch = f_name.split("_")[-1]
    #batch = batch.split(".")[-2]
    #batch_list.append(batch)
    #print(batch)

    if i == 0:  #create new dataframe in first iteration
        
        df = pd.read_csv(files)
        
        #the following lines can be uncommented to view bar plots of individual dataframes
        #print('\033[1m' + f"Barplot for {batch}" + '\033[0m')
        #im = plot_dist(df, "label_name", "bar")  #plot individual dataframes
        #display(im.figure)

    else:  #concatenate other csv's with first dataframe
        data = pd.read_csv(files)
        df = pd.concat([df, data], ignore_index=True)
    i += 1

In [5]:
df

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,mri,17,16,138,173,mildDem0.jpg,176,208
1,mri,16,17,140,172,mildDem1.jpg,176,208
2,mri,18,17,139,175,mildDem10.jpg,176,208
3,mri,18,17,138,172,mildDem100.jpg,176,208
4,mri,16,17,139,174,mildDem101.jpg,176,208
...,...,...,...,...,...,...,...,...
795,mri,16,16,142,174,verymildDem345.jpg,176,208
796,mri,17,17,140,174,verymildDem346.jpg,176,208
797,mri,16,17,140,174,verymildDem347.jpg,176,208
798,mri,17,17,140,172,verymildDem348.jpg,176,208


### Convert unique image names to list 

In [6]:
img_names = df["image_name"].unique().tolist()
len(img_names)

800

In [7]:
training = int(len(img_names) * 0.8)
valid = len(img_names) - training
print(training, valid)

640 160


### Extract random image names in a list length of validation number

In [8]:
val_list = []   #initialize empty list for storing random unique names for validation datset
ite = 0

#while loop to extract random unique names
while len(val_list) < valid:
    ran = random.randint(0, len(img_names)-1)  #generate random numbers from total length of unique names
    curr_name = img_names[ran]
    if len(val_list) == 0:  #save first name into list
        val_list.append(curr_name)
    elif len(val_list) != 0 and curr_name not in val_list:  #check if every other name does not exist before appending
        val_list.append(curr_name)
        
    ite += 1
    
print(ite)

175


#### Verify that only unique names are in list 

In [9]:
tem_df = pd.DataFrame(list(zip(val_list)), columns = ['names'])
tem_df

,names
0,verymildDem42.jpg
1,moderateDem40.jpg
2,verymildDem229.jpg
3,verymildDem100.jpg
4,mildDem308.jpg
...,...
155,mildDem303.jpg
156,mildDem185.jpg
157,moderateDem38.jpg
158,mildDem33.jpg


In [10]:
tem_df["names"].nunique()

160

### Create directories to store train and validation data if they do not exist

In [11]:
main_dir = "MRI_Verification_Data"
train_dir = "train"
val_dir = "valid"
img_dir = "images"
yolo_labels = "labels"
xml_labels = "labels_xml"
csv_labels = "labels_csv"

if os.path.isdir(f"{main_dir}"):
    #os.rmdir(f"{main_dir}")
    shutil.rmtree(f"{main_dir}")


if not os.path.isdir(f"{main_dir}"):
    os.makedirs(f"{main_dir}")
if not os.path.isdir(f"{main_dir}/{train_dir}/{img_dir}"):
    os.makedirs(f"{main_dir}/{train_dir}/{img_dir}")
if not os.path.isdir(f"{main_dir}/{train_dir}/{yolo_labels}"):
    os.makedirs(f"{main_dir}/{train_dir}/{yolo_labels}")
if not os.path.isdir(f"{main_dir}/{train_dir}/{xml_labels}"):
    os.makedirs(f"{main_dir}/{train_dir}/{xml_labels}")
if not os.path.isdir(f"{main_dir}/{train_dir}/{csv_labels}"):
    os.makedirs(f"{main_dir}/{train_dir}/{csv_labels}")
    
if not os.path.isdir(f"{main_dir}/{val_dir}/{img_dir}"):
    os.makedirs(f"{main_dir}/{val_dir}/{img_dir}")
if not os.path.isdir(f"{main_dir}/{val_dir}/{yolo_labels}"):
    os.makedirs(f"{main_dir}/{val_dir}/{yolo_labels}")
if not os.path.isdir(f"{main_dir}/{val_dir}/{xml_labels}"):
    os.makedirs(f"{main_dir}/{val_dir}/{xml_labels}")
if not os.path.isdir(f"{main_dir}/{val_dir}/{csv_labels}"):
    os.makedirs(f"{main_dir}/{val_dir}/{csv_labels}")

### Extract train and validation dataframe from main dataframe 

In [12]:
#Extract train dataset list
train_list = [n for n in img_names if n not in val_list]
len(train_list)

640

In [13]:
#extract train dataframe from total dataframe
df_train = df.query(f"image_name in {train_list}")
df_train

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,mri,17,16,138,173,mildDem0.jpg,176,208
1,mri,16,17,140,172,mildDem1.jpg,176,208
2,mri,18,17,139,175,mildDem10.jpg,176,208
4,mri,16,17,139,174,mildDem101.jpg,176,208
6,mri,18,17,138,173,mildDem103.jpg,176,208
...,...,...,...,...,...,...,...,...
795,mri,16,16,142,174,verymildDem345.jpg,176,208
796,mri,17,17,140,174,verymildDem346.jpg,176,208
797,mri,16,17,140,174,verymildDem347.jpg,176,208
798,mri,17,17,140,172,verymildDem348.jpg,176,208


In [14]:
#extract train dataframe from total dataframe
df_valid = df.query(f"image_name in {val_list}")
df_valid

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
3,mri,18,17,138,172,mildDem100.jpg,176,208
5,mri,17,17,138,173,mildDem102.jpg,176,208
15,mri,17,18,139,172,mildDem111.jpg,176,208
19,mri,15,16,139,173,mildDem115.jpg,176,208
30,mri,17,17,139,173,mildDem125.jpg,176,208
...,...,...,...,...,...,...,...,...
764,mri,16,17,141,173,verymildDem314.jpg,176,208
773,mri,16,17,140,173,verymildDem323.jpg,176,208
776,mri,16,17,140,173,verymildDem326.jpg,176,208
783,mri,16,17,141,173,verymildDem333.jpg,176,208


In [17]:
#verify that sum lengths of both dataframes are equal to length of total dataframe
(len(df_train) + len(df_valid))

800

In [18]:
#save both dataframeas as csv files in respective directories
df_train.to_csv(f"{main_dir}/{train_dir}/{csv_labels}/annotations.csv", index =  False)
df_valid.to_csv(f"{main_dir}/{val_dir}/{csv_labels}/annotations.csv", index =  False)


In [19]:
df_train.label_name.value_counts()

mri    640
Name: label_name, dtype: int64

In [20]:
df_valid.label_name.value_counts()

mri    160
Name: label_name, dtype: int64

### Use unique train and validation names to extract yolo files 

In [21]:
yolo_list = path_to_list(path_yolo, "txt")  #get list of files from yolo dirrectory

strip_train_list = []
for fl in train_list:
    
    if ".jpeg" in fl:
        #print("JPEG")
        strip_train_list.append(fl.split(".jpeg")[0])
    elif ".jpg" in fl:
        strip_train_list.append(fl.split(".jpg")[0])

ci = 0

#print(len(yolo_list))
for yls in yolo_list:
    f_name = os.path.split(yls)[-1]
    f = f_name.split(f".txt")[0]
    #print(f)
    if f in strip_train_list:
        shutil.copy(f"{yls}", f"{main_dir}/{train_dir}/{yolo_labels}/{f_name}")
        ci += 1
    
    #
print(f"Total number of train yolo text files are {ci}")

Total number of train yolo text files are 640


In [22]:
yolo_list = path_to_list(path_yolo, "txt")  #get list of files from yolo dirrectory

strip_val_list = []
for fl in val_list:
    #strip_val_list.append(fl.split(".jpg")[0])
    if ".jpeg" in fl:
        #print("JPEG")
        strip_val_list.append(fl.split(".jpeg")[0])
    elif ".jpg" in fl:
        strip_val_list.append(fl.split(".jpg")[0])

ci = 0

#print(len(yolo_list))
for yls in yolo_list:
    f_name = os.path.split(yls)[-1]
    f = f_name.split(f".txt")[0]
    #print(f)
    if f in strip_val_list:
        shutil.copy(f"{yls}", f"{main_dir}/{val_dir}/{yolo_labels}/{f_name}")
        ci += 1
    
    #
print(f"Total number of validation yolo text files are {ci}")

Total number of validation yolo text files are 160


### Use unique train and validation names to extract xml files  

In [23]:
"""xml_list = path_to_list(path_xml, "xml")  #get list of files from xml dirrectory


ci = 0

#print(len(yolo_list))
for yls in xml_list:
    f_name = os.path.split(yls)[-1]
    f = f_name.split(f".xml")[0]
    #print(f)
    #print(yls)
    if f in strip_train_list:
        shutil.copy(f"{yls}", f"{main_dir}/{train_dir}/{xml_labels}/{f_name}")
        ci += 1
    
    #
print(f"Total number of train xml text files are {ci}")"""

'xml_list = path_to_list(path_xml, "xml")  #get list of files from xml dirrectory\n\n\nci = 0\n\n#print(len(yolo_list))\nfor yls in xml_list:\n    f_name = os.path.split(yls)[-1]\n    f = f_name.split(f".xml")[0]\n    #print(f)\n    #print(yls)\n    if f in strip_train_list:\n        shutil.copy(f"{yls}", f"{main_dir}/{train_dir}/{xml_labels}/{f_name}")\n        ci += 1\n    \n    #\nprint(f"Total number of train xml text files are {ci}")'

In [24]:
"""xml_list = path_to_list(path_xml, "xml")  #get list of files from xml dirrectory


ci = 0

#print(len(yolo_list))
for yls in xml_list:
    f_name = os.path.split(yls)[-1]
    f = f_name.split(f".xml")[0]
    #print(f)
    #print(yls)
    if f in strip_val_list:
        shutil.copy(f"{yls}", f"{main_dir}/{val_dir}/{xml_labels}/{f_name}")
        ci += 1
    
    #
print(f"Total number of validation xml text files are {ci}")"""

'xml_list = path_to_list(path_xml, "xml")  #get list of files from xml dirrectory\n\n\nci = 0\n\n#print(len(yolo_list))\nfor yls in xml_list:\n    f_name = os.path.split(yls)[-1]\n    f = f_name.split(f".xml")[0]\n    #print(f)\n    #print(yls)\n    if f in strip_val_list:\n        shutil.copy(f"{yls}", f"{main_dir}/{val_dir}/{xml_labels}/{f_name}")\n        ci += 1\n    \n    #\nprint(f"Total number of validation xml text files are {ci}")'

### Extract images using unique image names

In [25]:
cleaned_csv_trackss = []
i = 0  #variable for keeping track of first dataframe in loop before concatenating others

total_img =  total_csvu = cleaned_xml = 0

train_exists_list = []

tb = 0

for n in range(1, 200):
    cleaned_csv_track = [] #for keeping track of cleaned csv files per batch
    if n < 10:
        bat = f"batch00{n}"
    elif n >= 10 and n < 100:
        bat = f"batch0{n}"
    else:
        bat = f"batch{n}"

    
    path_img_jpeg = f"{pat}/{bat}/*.jpeg"  # Match jpeg files
    jpeg_files = glob.glob(path_img_jpeg)
    
    path_img = f"{pat}/{bat}/*.jpg"  # Match jpg files
    jpg_files = glob.glob(path_img)

    # Combine the lists of jpg and jpeg files
    all_files = jpg_files + jpeg_files
    #print(len(jpeg_files))
    #print(len(jpg_files), len(jpeg_files), len(all_files))

    
    curr_bat_img = 0
    for files in all_files:  #iterate through folder to filter images
        f_name = os.path.split(files)[-1]
        if ".jpeg" in f_name:
            #print("JPEG")
            f = f_name.split(f".jpeg")[0]
            if f in strip_train_list:
                #print(files)
                i += 1
                if len(train_exists_list) == 0:
                    train_exists_list.append(f)
                    shutil.copy(f"{files}", f"{main_dir}/{train_dir}/{img_dir}/{f}.jpeg")
                else:
                    if f not in train_exists_list:
                        train_exists_list.append(f)
                        shutil.copy(f"{files}", f"{main_dir}/{train_dir}/{img_dir}/{f}.jpeg")
            
        elif ".jpg" in f_name:
            f = f_name.split(f".jpg")[0]
            if f in strip_train_list:
                i += 1
                if len(train_exists_list) == 0:
                    train_exists_list.append(f)
                    shutil.copy(f"{files}", f"{main_dir}/{train_dir}/{img_dir}/{f}.jpg")
                else:
                    if f not in train_exists_list:
                        train_exists_list.append(f)
                        shutil.copy(f"{files}", f"{main_dir}/{train_dir}/{img_dir}/{f}.jpg")
        #i += 1

    
#print(f"Total number of training images are {i}")
print(f"Total number of training images are {len(train_exists_list)}")

Total number of training images are 640


In [26]:
i = 0  #variable for keeping track of first dataframe in loop before concatenating others

total_img =  total_csvu = cleaned_xml = 0
exists_list = [] 



for n in range(1, 200):
    cleaned_csv_track = [] #for keeping track of cleaned csv files per batch
    if n < 10:
        bat = f"batch00{n}"
    elif n >= 10 and n < 100:
        bat = f"batch0{n}"
    else:
        bat = f"batch{n}"

    
    
    path_img = f"{pat}/{bat}/*.jpg"  # Match jpg files
    jpg_files = glob.glob(path_img)

    path_img = f"{pat}/{bat}/*.jpeg"  # Match jpeg files
    jpeg_files = glob.glob(path_img)

    # Combine the lists of jpg and jpeg files
    all_files = jpg_files + jpeg_files

    #path_img = f"{pat}/{bat}/*?.jpg"
    curr_bat_img = 0
    for files in all_files:  #iterate through folder to filter images
        #print(files)
        f_name = os.path.split(files)[-1]
        f = f_name.split(f".jpg")[0]

        #print(f)
        if ".jpg" in f_name:
            #print("JPG")
            f = f_name.split(f".jpg")[0]
            if f in strip_val_list:
                #shutil.copy(f"{files}", f"{main_dir}/{val_dir}/{img_dir}/{f}.jpg")
                #shutil.copy(f"{yls}", f"{main_dir}/{val_dir}/{img_dir}/{f_name}")
                if len(exists_list) == 0:
                    exists_list.append(f)
                    shutil.copy(f"{files}", f"{main_dir}/{val_dir}/{img_dir}/{f}.jpg")
                else:
                    if f not in exists_list:
                        exists_list.append(f)
                        shutil.copy(f"{files}", f"{main_dir}/{val_dir}/{img_dir}/{f}.jpg")
            
        elif ".jpeg" in f_name:
            #print("JPEG")
            f = f_name.split(f".jpeg")[0]
            if f in strip_val_list:
                #shutil.copy(f"{files}", f"{main_dir}/{val_dir}/{img_dir}/{f}.jpg")
                #shutil.copy(f"{yls}", f"{main_dir}/{val_dir}/{img_dir}/{f_name}")
                if len(exists_list) == 0:
                    exists_list.append(f)
                    shutil.copy(f"{files}", f"{main_dir}/{val_dir}/{img_dir}/{f}.jpeg")
                else:
                    if f not in exists_list:
                        exists_list.append(f)
                        shutil.copy(f"{files}", f"{main_dir}/{val_dir}/{img_dir}/{f}.jpeg")
            
            
            #i += 1

#print(i)
                
#print(f"Total number of validation images are {i}")
print(f"Total number of training images are {len(exists_list)}")

Total number of training images are 160


In [27]:
len(train_exists_list) + len(exists_list)

800